Initialize workflow.

In [44]:
import struct, socket
import shutil
import numpy as np
import pandas as pd
import linecache, bisect
import csv
import operator
import os
from IPython.html import widgets
from IPython.html.widgets import interact, interactive, fixed
from IPython.display import display

!source /etc/oni.conf
!export LUSER

spath = os.environ['LUSER'] + '/ipython/user/20150423/'
sconnect = spath + 'lda_scores.csv'
sconnectbu = spath + 'lda_scores_bu.csv'
tmpconnect = sconnect+'.tmp'
stemp = sconnect + '.new'
coff = 150;
contxtpath =  os.environ['HUSER'] + '/ipython/iploc/'
nwloc = contxtpath + 'networkcontext.txt'
iploc = contxtpath + 'iploc.csv'

print 'loading context data ... may take a few minutes.'
iplist = np.loadtxt(iploc,dtype=np.uint32,delimiter=',',usecols={0},
                    converters={0: lambda s: np.uint32(s.replace('"',''))})
srcdict,srclist = {},[]
dstdict,dstlist = {},[]
sportdict,sportlist = {},[]
dportdict,dportlist = {},[]
print 'modules and data loaded'

loading context data ... may take a few minutes.
modules and data loaded


In [45]:
#load LDA scores#update lda doc with scores - either one edge,# or ALL that contain x IP and y port
def displaythis() :
    # build dict of IP addresses
    #sev,score, tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt
    srcdict,srclist = {},[]
    dstdict,dstlist = {},[]
    sportdict,sportlist = {},[]
    dportdict,dportlist = {},[]
    srclist.append('')
    dstlist.append('')
    sportlist.append('')
    dportlist.append('')
    with open(sconnect, 'r') as f:
        reader = csv.reader(f,delimiter=',') 
        reader.next();
        rowct = 1
        for row in reader:
            if row[2] not in srcdict:
                srclist.append(row[2])
                srcdict[row[2]] = struct.unpack("!L", socket.inet_aton(row[2]))[0]
            if row[3] not in dstdict:
                dstlist.append(row[3])
                dstdict[row[3]] = struct.unpack("!L", socket.inet_aton(row[3]))[0]
            if row[4] not in sportdict:
                sportlist.append(row[4])
                sportdict[row[4]] = row[4]
            if row[5] not in dportdict:
                dportlist.append(row[5])
                dportdict[row[5]] = row[5]
            if rowct == coff:
                break;
            rowct += 1    
    searches = widgets.Box(width=550,height=200)
    searchp = widgets.HBox(width=550,height=150)
    actions = widgets.Box(width=50,height=150)
    srcselect = widgets.Select(options=srclist,height=min(len(srclist)*10,150),width=120)
    dstselect = widgets.Select(options=dstlist,height=min(len(dstlist)*10,150),width=120)
    sportselect = widgets.Select(options=sportlist,height=min(len(sportlist)*10,150),width=75)
    dportselect = widgets.Select(options=dportlist,height=min(len(dportlist)*10,150),width=75)
    tableHTML = ("<table width='100%'><tr><td>" +
    "<td>Source IP:</td><td>Dest IP:</td><td>Src Port</td><td>Dst Port:</td></tr></table>")
    srctext = widgets.Text(value='',width=150, placeholder='Quick IP scoring')
    toplabel = widgets.HTML(value=tableHTML,width=550)
    ratingbut = widgets.RadioButtons(description='Rating:',options=['1', '2', '3'],width=50)
    assignbut = widgets.Button(description='Assign')
    updatebut = widgets.Button(description='Update')
    actions.children = [srctext,ratingbut,assignbut,updatebut]
    searchp.children = [srcselect,dstselect,sportselect,dportselect,actions]
    searches.children = [toplabel,searchp]
    display(searches)
    
    def update_sconnects(b):
        #define logic based on combo of input
        if srctext.value != '':
            svals = [srctext.value,dstselect.value,sportselect.value,dportselect.value]
        else:
            svals = [srcselect.value,dstselect.value,sportselect.value,dportselect.value]
        risk = ratingbut.value
        shash = 0
        fhash = [2,3,4,5,12]
        for k in xrange(len(svals)):
            if svals[k] != '':                
                shash += 2**k
        with open(tmpconnect,'w') as g:
            riter = csv.writer(g,delimiter=',')
            riter.writerow(["sev"]+["tstart"]+["srcIP"]+["dstIP"]+["sport"]+
                           ["dport"]+["proto"]+["ipkt"]+["ibyt"]+["lda_score"]+["srcIpInternal"]+["destIpInternal"]
                           +["srcGeo"]+["dstGeo"]+["srcDomain"]+["dstDomain"])
            with open(sconnect, 'r') as f:
                reader = csv.reader(f,delimiter=',')
                reader.next();
                rowct = 1
                #sev,tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt,lda_score
                for row in reader: 
                    result = 0
                    for n in xrange(0,5):
                        if (2**n & shash) > 0:                        
                            if row[fhash[n]] == svals[n]:
                                result += 2**n
                    if result == shash:
                        row[0] = risk
                        print row
                    riter.writerow(row) 
                    rowct += 1
        shutil.copyfile(tmpconnect,sconnect)
            
    def savesort(b):
        with open(stemp,'w') as g:
            riter = csv.writer(g,delimiter=',')
            riter.writerow(["sev"]+["tstart"]+["srcIP"]+["dstIP"]+["sport"]+["dport"]+["proto"]+["ipkt"]+["ibyt"]+["lda_score"]
                           +["srcIpInternal"]+["destIpInternal"] +["srcGeo"]+["dstGeo"]+["srcDomain"]+["dstDomain"])
            reader = csv.reader(open(sconnect), delimiter=",")
            reader.next()
            srtlist = sorted(reader, key=lambda x: (int(x[0]), float(x[9])))
            riter.writerows(srtlist)
        #!mv $stemp $sconnect
        shutil.copyfile(stemp,sconnect)

    assignbut.on_click(update_sconnects)
    updatebut.on_click(savesort)


#bisect - linecache
# iterate through IP addresses and attach ip location info

# print IP addresses
def geoip() :
    srcdict,srclist = {},[]
    dstdict,dstlist = {},[]
    sportdict,sportlist = {},[]
    dportdict,dportlist = {},[]
    srclist.append('')
    dstlist.append('')
    sportlist.append('')
    dportlist.append('')
    with open(sconnect, 'r') as f:
        reader = csv.reader(f,delimiter=',') 
        reader.next();
        rowct = 1
        for row in reader:
            if row[2] not in srcdict:
                srclist.append(row[2])
                srcdict[row[2]] = struct.unpack("!L", socket.inet_aton(row[2]))[0]
            if row[3] not in dstdict:
                dstlist.append(row[3])
                dstdict[row[3]] = struct.unpack("!L", socket.inet_aton(row[3]))[0]
            if row[4] not in sportdict:
                sportlist.append(row[4])
                sportdict[row[4]] = row[4]
            if row[5] not in dportdict:
                dportlist.append(row[5])
                dportdict[row[5]] = row[5]
            if rowct == coff:
                break;
            rowct += 1    
            linecache.clearcache()
    for srcip in srcdict:
        srcdict[srcip] = linecache.getline(iploc, bisect.bisect(iplist,srcdict[srcip])).replace('\n','')
    for dstip in dstdict:
        dstdict[dstip] = linecache.getline(iploc, bisect.bisect(iplist,dstdict[dstip])).replace('\n','')
    with open(sconnect, 'r') as f:
        reader = csv.reader(f,delimiter=',') 
        reader.next();
        rowct = 1
        for row in reader:
            #print "line",rowct,row,":"
            if srcdict[row[2]] == '':
                srcip = 'unknown'
            else:
                srcip = srcdict[row[2]]
            if dstdict[row[3]] == '':
                dstip = 'unknown'
            else:
                dstip = dstdict[row[3]]
            srcgeoip = srcip.replace('"','').split(',')
            dstgeoip = dstip.replace('"','').split(',')
            
            print 'row',rowct,':',row[2],srcgeoip[4:6],srcgeoip[8:10],"-->",row[3],dstgeoip[4:6],dstgeoip[8:10]
            if rowct == 100:
                break
            rowct += 1

def set_rules():
    rops = ['leq','leq','leq','leq','leq','leq']
    rvals = ['','','',1024,'',54]
    risk = 2
    apply_rules(rops,rvals,risk)
    rops = ['leq','leq','leq','leq','eq','eq']
    rvals = ['','','',1024,3,152]
    risk = 2
    apply_rules(rops,rvals,risk)
    rops = ['leq','leq','leq','leq','eq','eq']
    rvals = ['','','',1024,2,104]
    risk = 2
    rops = ['leq','leq','eq','leq','leq','leq']
    rvals = ['','',0,1023,'','']
    risk = 2

    apply_rules(rops,rvals,risk)

def apply_rules(rops,rvals,risk):
    #define logic based on combo of input
    rhash = 0
    rfhash = [2,3,4,5,7,8]
    for k in xrange(len(rvals)):
        if rvals[k] != '':                
            rhash += 2**k
    with open(tmpconnect,'w') as g:
        riter = csv.writer(g,delimiter=',')
        riter.writerow(["sev"]+["tstart"]+["srcIP"]+["dstIP"]+["sport"]+
                       ["dport"]+["proto"]+["ipkt"]+["ibyt"]+["lda_score"]+["srcIpInternal"]+["destIpInternal"]
                      +["srcGeo"]+["dstGeo"]+["srcDomain"]+["dstDomain"])
        with open(sconnect, 'r') as f:
            reader = csv.reader(f,delimiter=',')
            reader.next();
            rowct = 1
            #sev,tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt,lda_score
            for row in reader: 
                result = 0
                for n in xrange(0,6):
                    if (2**n & rhash) > 0:
                        if rops[n] == 'leq':
                            if int(row[rfhash[n]]) <= int(rvals[n]):
                                result += 2**n                           
                        if rops[n] == 'eq':
                            if int(row[rfhash[n]]) == int(rvals[n]):
                                result += 2**n                           
                if result == rhash:
                    row[0] = risk
                    print row
                    rowct += 1
                riter.writerow(row) 
    print 'rows applied', rowct         
    shutil.copyfile(tmpconnect,sconnect)

def attack_heuristics():
    with open(sconnect, 'r') as f:
        reader = csv.reader(f,delimiter=',') 
        reader.next();
        rowct = 1
        for row in reader:
            if row[2] not in srcdict:
                srcdict[row[2]] = row[2]
            if row[3] not in dstdict:
                 dstdict[row[3]] = row[3]
            if row[4] not in sportdict:
                sportdict[row[4]] = row[4]
            if row[5] not in dportdict:
                dportdict[row[5]] = row[5]


    df = pd.read_csv(sconnect)
    
   
    #print df

    gb = df.groupby([u'srcIP'])
    
    #print gb.groups.keys()
    
    for srcip in srcdict:
        if len(gb.get_group(srcip)) > 20:
            print srcip,'connects:',len(gb.get_group(srcip))


Complete batch risk scoring according to rules.

In [46]:
set_rules()

[2, '4/23/2015 7:41', '171.77.205.38', '134.134.139.74', '55338', '445', 'TCP', '1', '52', '1.40E-09', '0', '0', 'Maharashtra;Mumbai Bharti Airtel Limited', 'Oregon;Hillsboro Intel Corporation', 'airtel.in', 'intel.co', '0', '1', 'Maharashtra;Mumbai Bharti Airtel Limited', 'Oregon;Hillsboro Intel Corporation', 'airtel.in', 'intel.com']
[2, '4/23/2015 23:41', '208.187.135.2', '134.134.139.78', '32845', '445', 'TCP', '1', '52', '1.49E-09', '0', '0', 'Washington;Vancouver Integra Telecom Inc.', 'Oregon;Hillsboro Intel Corporation', 'integratelecom.com', 'intel.co', '0', '1', 'Washington;Vancouver Integra Telecom Inc.', 'Oregon;Hillsboro Intel Corporation', 'integratelecom.com', 'intel.com']
[2, '4/23/2015 10:26', '66.240.192.138', '134.134.139.74', '63487', '129', 'UDP', '1', '29', '1.50E-09', '0', '0', 'California;San Diego Carinet Inc.', 'Oregon;Hillsboro Intel Corporation', 'sysadminbootcamp.com', 'intel.co', '0', '1', 'California;San Diego Carinet Inc.', 'Oregon;Hillsboro Intel Corpor

Run attack heuristics.

In [48]:
attack_heuristics()

KeyError: '134.134.248.227'

In [ ]:
displaythis()
geoip()

In [ ]:
#!cp $sconnectbu $sconnect